### MLFLOW integration with DagsHub

In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'g:\\Il mio Drive\\PROJECTS\\Kidney-Tumor-Detection-End-To-End'

Collect MLFlow url and for security reasons pasted it into .env file. Instructions below

Dagshub -> add repo -> remote -> experiments -> copy "using MLFlow tracking"

- MLFLOW_TRACKING_URI=https://dagshub.com/DavidAlexanderMoe/Kidney-Tumor-Detection-End-To-End.mlflow
- MLFLOW_TRACKING_USERNAME=DavidAlexanderMoe
- MLFLOW_TRACKING_PASSWORD=d8b4e71758deec52ce96b8105652ddaf6bad36aa
- python script.py

If someone has this info, he can send his experiments directly to the DagsHub repository.

git bash:
- export MLFLOW_TRACKING_URI=https://dagshub.com/DavidAlexanderMoe/Kidney-Tumor-Detection-End-To-End.mlflow
- export MLFLOW_TRACKING_USERNAME=DavidAlexanderMoe
- MLFLOW_TRACKING_PASSWORD=...

this info will be required in my main.py

#### for remote server only (remote) add this code
- remote_server_uri=https://dagshub.com/DavidAlexanderMoe/Kidney-Tumor-Detection-End-To-End.mlflow
- mlflow.set_tracking_uri(remote_server_uri)

git bash -> python main.py -> mlruns folder will be created

- go to dagshub -> experiments -> click on experiment
or
- go to dagshub -> remote -> experiments -> go to mlflow ui -> mlflow opens (running on local server)

run main.py to create another experiment

This is powerful since many different people can run different experiments on the same project and track them all in a single dashboard

Model monitoring -> different models and versions and decide which to push into production and which is staging for further tuning

#### Get the info and change environment:

In [4]:
from dotenv import load_dotenv
load_dotenv()

# getenv() -> read the value of the environment variable
MLFLOW_TRACKING_URI = os.getenv("MLFLOW_TRACKING_URI")
MLFLOW_TRACKING_USERNAME = os.getenv("MLFLOW_TRACKING_USERNAME")
MLFLOW_TRACKING_PASSWORD = os.getenv("MLFLOW_TRACKING_PASSWORD")

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/DavidAlexanderMoe/Kidney-Tumor-Detection-End-To-End.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]=MLFLOW_TRACKING_USERNAME
os.environ["MLFLOW_TRACKING_PASSWORD"]=MLFLOW_TRACKING_PASSWORD

In [6]:
import tensorflow as tf

In [7]:
# load model
model = tf.keras.models.load_model("artifacts/training/model.h5")

#### Update entity

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict            # from params.yaml
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

#### Model Evaluation Configuration

In [9]:
from CNN_Classifier.constants import *
from CNN_Classifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Kidney Images",
            mlflow_uri="https://dagshub.com/DavidAlexanderMoe/Kidney-Tumor-Detection-End-To-End.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

#### Update components

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras         # keras support for mlflow
from urllib.parse import urlparse

In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    # make again a valid set generator to separate out my testing data using keras
    # copied and pasted from keras documentation
    # basically takes my data in artifacts and then prepare TESTING data for me
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30       # now 0.3
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    # same thing -> use staticmethod to make this function indipendent and not depend on self and the class
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    # evaluate model
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()           # to create a json file with the scores
    
    # save metrics
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    
    # Experiment tracking and model registration with mlflowù
    # copied from mlflow doc and modified for logging
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():            # start tracking
            mlflow.log_params(self.config.all_params)           # log parameters and metrics
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]})
            
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

#### update pipeline

In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2023-11-18 14:18:08,949: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-18 14:18:08,973: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-18 14:18:08,973: INFO: common: created directory at: artifacts]


Found 2207 images belonging to 2 classes.
138/138 [==============================] - 289s 2s/step - loss: 4.2983 - accuracy: 0.7300
[2023-11-18 14:22:59,372: INFO: common: json file saved at: scores.json]


2023/11/18 14:23:05 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2023-11-18 14:23:08,396: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\david\AppData\Local\Temp\tmpvc1wr8e0\model\data\model\assets
[2023-11-18 14:23:09,494: INFO: builder_impl: Assets written to: C:\Users\david\AppData\Local\Temp\tmpvc1wr8e0\model\data\model\assets]


c:\Users\david\miniconda3\envs\kidney\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2023/11/18 14:24:28 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.


Saved json with scores

Go to MLFlow -> refresh -> see the experiment

Now the point of implementing mlflow tracking: If i am not satisfied with the results (metrics), i can easily change my hyperparameters through the params.yaml file without the need of doing hyperparameter tuning manually. I must run the entire pipeline again (via main.py) since the model.h5 was trained with different parameters.

MLFlow -> models -> see model registration. Here you test, test, test and then, when satisfied with the results, you can move the model into production stage by clicking the version of the model and changing the stage.

---

- Changed epochs from 1 to 2 to see a boost in accuracy, open main.py and run another experiment and see the Version 2 of the model.

Run many experiments (lr=0.08, augmentation=false, change batch size, etc.. )

Compare the experiments/models directly on mlflow ui through dagshub.

## Once the pipepline and the main.py are updated, you can main.py different times with different parameters setting them in params.yaml!